# 高度なディープラーニングのベストプラクティス

- Keras Functional API    
- Keras のコールバックの使用    
- TensorBoardの操作    
- 最先端のモデルを開発するための重要なベストプラクティス    

In [11]:
!pip list | grep Keras

Keras                    2.1.6      


# 多入力モデル

In [1]:
from keras.models import Model
from keras import layers
from keras import Input

text_vocabulary_size = 10000
question_vocabulary_size = 10000
answer_vocabulary_size = 500

text_input = Input(shape=(None,), dtype='int32', name='text')

embedded_text = layers.Embedding(text_vocabulary_size, 64)(text_input)
encoded_text = layers.LSTM(32)(embedded_text)

question_input = Input(shape=(None,), dtype='int32', name='question')
embedded_question = layers.Embedding(question_vocabulary_size, 32)(question_input)
encoded_question = layers.LSTM(16)(embedded_question)

concatenated = layers.concatenate([encoded_text, encoded_question], axis=-1)

answer = layers.Dense(answer_vocabulary_size, activation='softmax')(concatenated)

model = Model([text_input, question_input], answer)
model.compile(optimizer='rmsprop',
             loss='categorical_crossentropy',
             metrics=['acc'])

Using TensorFlow backend.


In [2]:
#多入力モデルへのデータの供給
import numpy as np
num_samples = 1000
max_length = 100

text = np.random.randint(1, text_vocabulary_size,
                        size=(num_samples, max_length))
question = np.random.randint(1, question_vocabulary_size,
                            size=(num_samples, max_length))

answers = np.zeros(shape=(num_samples, answer_vocabulary_size))
indices = np.random.randint(0, answer_vocabulary_size, size=num_samples)
for i, x in enumerate(answers):
    x[indices[i]] = 1

#入力リストを使った適応
model.fit([text, question], answers, epochs=10, batch_size=128)



Epoch 1/10
1000/1000 [==============================] - 5s 5ms/step - loss: 6.2145 - acc: 0.0020
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.1991 - acc: 0.0480
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.1665 - acc: 0.0190
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.0842 - acc: 0.0090
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/step - loss: 6.0232 - acc: 0.0090
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.9682 - acc: 0.0100
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.8789 - acc: 0.0110
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.7739 - acc: 0.0130
Epoch 9/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.6948 - acc: 0.0230
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.6014 - acc: 0.0280


In [3]:
#入力ディクショナリを使用
model.fit({'text': text, 'question': question}, answers, epochs=10, batch_size=128)


Epoch 1/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.5555 - acc: 0.0390
Epoch 2/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.4557 - acc: 0.0470
Epoch 3/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.4027 - acc: 0.0480
Epoch 4/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.3162 - acc: 0.0550
Epoch 5/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.2684 - acc: 0.0460
Epoch 6/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.2217 - acc: 0.0550
Epoch 7/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.1511 - acc: 0.0590
Epoch 8/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.0820 - acc: 0.0670
Epoch 9/10
1000/1000 [==============================] - 3s 3ms/step - loss: 5.0241 - acc: 0.0690
Epoch 10/10
1000/1000 [==============================] - 3s 3ms/step - loss: 4.9691 - acc: 0.0780


# 多出力モデル

In [0]:
from keras import layers
from keras import Input
from keras.models import Model

vocabulary_size = 500
num_income_groups = 10

posts_input = Input(shape=(None,) ,dtype='int32', name='posts')

embedded_posts = layers.Embedding(256, vocabulary_size)(posts_input)
x = layers.Conv1D(128, 5, activation='relu', name='conv1')(embedded_posts)
x = layers.MaxPooling1D(5, name='pool1')(x)
x = layers.Conv1D(256, 5, activation='relu', name='conv2')(x)
x = layers.Conv1D(256, 5, activation='relu', name='conv3')(x)
#入力サイズ的にここまで畳み込めない
#各層に名前をつけて、どの層でエラーになってるか見ること
# x = layers.MaxPooling1D(5, name='pool2')(x)
# x = layers.Conv1D(256, 5, activation='relu', name='conv4')(x)
# x = layers.Conv1D(256, 5, activation='relu', name='conv5')(x)
x = layers.GlobalMaxPooling1D(name='gcp1')(x)
x = layers.Dense(128, activation='relu', name='fc1')(x)

age_prediction = layers.Dense(1, name='age')(x)
income_prediction = layers.Dense(num_income_groups,
                                activation='softmax',
                                name='income')(x)
gender_prediction = layers.Dense(1, activation='sigmoid', name='gender')(x)
model = Model(posts_input,
             [age_prediction, income_prediction, gender_prediction])

In [0]:
model.compile(optimizer='rmsprop',
             loss=['mse',
                  'categorical_crossentropy',
                  'binary_crossentropy'])

In [0]:
#多入力モデルへのデータの供給
import numpy as np
num_samples = 1000

posts = np.random.randint(1, size=(num_samples, 200))
age_targets = np.random.randint(1, size=(num_samples, 1))
income_targets = np.random.randint(1, size=(num_samples, 10))
gender_targets = np.random.randint(1, size=(num_samples, 1))


In [35]:
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=1, batch_size=128)


Epoch 1/1
1000/1000 [==============================] - 3s 3ms/step - loss: 0.6222 - age_loss: 0.3805 - income_loss: 0.0000e+00 - gender_loss: 0.2417


In [0]:
model.compile(optimizer='rmsprop',
             loss={'age': 'mse',
                   'income': 'categorical_crossentropy',
                   'gender': 'binary_crossentropy'
             })

In [39]:
model.fit(posts, {'age': age_targets, 'income': income_targets, 'gender': gender_targets}, epochs=1, batch_size=128)


Epoch 1/1
1000/1000 [==============================] - 1s 554us/step - loss: 0.0127 - age_loss: 0.0094 - income_loss: 0.0000e+00 - gender_loss: 0.0033


損失関数の貢献度が不均衝(ふきんこう)である場合はモデルの表現は最も大きい損失値を持つタスクを優先する形で最適化される。    

この問題を解決するには損失に重要度を割り当てる。

In [0]:
model.compile(optimizer='rmsprop',
             loss=['mse',
                  'categorical_crossentropy',
                  'binary_crossentropy'],
             loss_weights=[0.25, 1., 10.])

In [41]:
#入力リストを使った適応
model.fit(posts, [age_targets, income_targets, gender_targets], epochs=1, batch_size=128)


Epoch 1/1
1000/1000 [==============================] - 1s 1ms/step - loss: 0.1043 - age_loss: 0.4090 - income_loss: 0.0000e+00 - gender_loss: 2.0175e-04


In [0]:
model.compile(optimizer='rmsprop',
             loss={'age': 'mse',
                   'income': 'categorical_crossentropy',
                   'gender': 'binary_crossentropy'},
             loss_weights={'age':0.25 ,'income': 1.,'gender': 10.})

In [43]:
model.fit(posts, {'age': age_targets, 'income': income_targets, 'gender': gender_targets}, epochs=1, batch_size=128)


Epoch 1/1
1000/1000 [==============================] - 2s 2ms/step - loss: 0.1573 - age_loss: 0.6278 - income_loss: 0.0000e+00 - gender_loss: 4.0115e-05


# 層の有向非巡回グラフ


###Inceptionモジュール

In [49]:

from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

11493376/11490434 [==============================] - 0s 0us/step


In [75]:
print(train_images.shape)
print(test_labels.shape)

(60000, 28, 28, 1)
(10000, 10)


In [79]:
from keras.models import Model
from keras import Input
from keras import layers
from keras import models
num_classes = 10
# img_input = Input(shape=train_images.shape[1:])
img_input = Input(shape=(28, 28, 1))
x = layers.Conv2D(32, (3, 3), activation='relu', padding='same', name='conv_input')(img_input)
# model = models.Sequential()
# model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 1)))

#縦横フィルター計算式
#OH: (H+2P-FH/S) + 1
#OW: (W+2P-FW/S) + 1
#H:入力縦幅#P:パディング#FH:フィルター縦幅#S:ストライド

#OH:(28+2・0-1/2)+1=14.5
branch_a = layers.Conv2D(128, 1, activation='relu', padding='same', strides=2, name="conv_a1")(x)

#OH:(28+2・0-1/1)+1=28
branch_b = layers.Conv2D(128, 1, activation='relu', padding='same', name="conv_b1")(x)
#OH:(28+2・0-3/2)+1=13.5
branch_b = layers.Conv2D(128, 3, activation='relu', padding='same', strides=2, name="conv_b2")(branch_b)

#OH:(28+2・0-3/2)+1=13.5
branch_c = layers.AveragePooling2D(3, strides=2, padding='same', name="apool_c1")(x)
#OH:(13.5+2・0-3/1)+1=11.5
branch_c = layers.Conv2D(128, 3, activation='relu', padding='same', name="conv_c1")(branch_c)

branch_d = layers.Conv2D(128, 1, activation='relu', padding='same', name="conv_d1")(x)
branch_d = layers.Conv2D(128, 3, activation='relu', padding='same', name="conv_d2")(branch_d)
branch_d = layers.Conv2D(128, 3, activation='relu', padding='same', strides=2, name="conv_d3")(branch_d)

#shapeが違うとエラー
#(None, 14, 14, 32), (None, 13, 13, 32), (None, 11, 11, 32), (None, 12, 12, 32)
output = layers.concatenate([branch_a, branch_b, branch_c, branch_d], axis=-1)
print(output)
x = layers.Conv2D(32, (3, 3), activation='relu', name='conv_output')(output)
print(x)
x = layers.Flatten()(x)
x = layers.Dense(num_classes)(x)
x = layers.Dense(num_classes, activation='softmax', name='predictions')(x)
model = Model(img_input, [x], name='')

Tensor("concatenate_15/concat:0", shape=(?, 14, 14, 512), dtype=float32)
Tensor("conv_output_6/Relu:0", shape=(?, 12, 12, 32), dtype=float32)


In [80]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=5, batch_size=64)

Epoch 1/5
60000/60000 [==============================] - 70s 1ms/step - loss: 0.1699 - acc: 0.9498
Epoch 2/5
 3776/60000 [>.............................] - ETA: 1:02 - loss: 0.0506 - acc: 0.9831

60000/60000 [==============================] - 67s 1ms/step - loss: 0.0466 - acc: 0.9857
Epoch 3/5
 9024/60000 [===>..........................] - ETA: 56s - loss: 0.0314 - acc: 0.9891

60000/60000 [==============================] - 67s 1ms/step - loss: 0.0333 - acc: 0.9899
Epoch 4/5
11456/60000 [====>.........................] - ETA: 54s - loss: 0.0227 - acc: 0.9924

60000/60000 [==============================] - 67s 1ms/step - loss: 0.0243 - acc: 0.9927
Epoch 5/5
12608/60000 [=====>........................] - ETA: 53s - loss: 0.0168 - acc: 0.9944

60000/60000 [==============================] - 67s 1ms/step - loss: 0.0195 - acc: 0.9940


# 残差接続

In [1]:

from keras.datasets import mnist
from keras.utils import to_categorical

(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

train_images = train_images.reshape((60000, 28, 28, 1))
train_images = train_images.astype('float32') / 255

test_images = test_images.reshape((10000, 28, 28, 1))
test_images = test_images.astype('float32') / 255

train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

Using TensorFlow backend.


11493376/11490434 [==============================] - 3s 0us/step


In [0]:
from keras.models import Model
from keras import Input
from keras import layers
from keras import models
num_classes = 10

img_input = Input(shape=(28, 28, 1))
x = layers.Conv2D(128, 3, activation='relu', padding='same', name='conv_input')(img_input)
y = layers.Conv2D(128, 3, activation='relu', padding='same', name="conv_1")(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same', name="conv_2")(y)
y = layers.Conv2D(128, 3, activation='relu', padding='same', name="conv_3")(y)
#残差
y = layers.add([y, x])

x = layers.Flatten()(y)
x = layers.Dense(num_classes)(x)
x = layers.Dense(num_classes, activation='softmax', name='predictions')(x)
model = Model(img_input, [x], name='')

In [3]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=1, batch_size=64)

Epoch 1/1
60000/60000 [==============================] - 73s 1ms/step - loss: 0.1644 - acc: 0.9548


importの仕方を変えた

In [0]:
from keras.models import Model
from keras import Input
from keras.layers import Conv2D, Add, Flatten, Dense
from keras import models
num_classes = 10

img_input = Input(shape=(28, 28, 1))
x = Conv2D(128, 3, activation='relu', padding='same', name='conv_input')(img_input)
y = Conv2D(128, 3, activation='relu', padding='same', name="conv_1")(x)
y = Conv2D(128, 3, activation='relu', padding='same', name="conv_2")(y)
y = Conv2D(128, 3, activation='relu', padding='same', name="conv_3")(y)
#残差
y = Add()([y, x])

x = Flatten()(y)
x = Dense(num_classes)(x)
x = Dense(num_classes, activation='softmax', name='predictions')(x)
model = Model(img_input, [x], name='')

In [7]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=1, batch_size=64)

Epoch 1/1
60000/60000 [==============================] - 71s 1ms/step - loss: 0.1737 - acc: 0.9505


In [8]:
from keras.models import Model
from keras import Input
from keras import layers
from keras import models
num_classes = 10

img_input = Input(shape=(28, 28, 1))
x = layers.Conv2D(128, 3, activation='relu', padding='same', name='conv_input')(img_input)
y = layers.Conv2D(128, 3, activation='relu', padding='same', name="conv_1")(x)
y = layers.Conv2D(128, 3, activation='relu', padding='same', name="conv_2")(y)
y = layers.MaxPooling2D(2, strides=2)(y)
print(y)
#14,14,128と同じにする1*1convを使った線形ダウンサンプリング
residual = layers.Conv2D(128, 1, strides=2, padding='same')(x)
print(residual)
#残差
y = layers.add([y, residual])

x = layers.Flatten()(y)
x = layers.Dense(num_classes)(x)
x = layers.Dense(num_classes, activation='softmax', name='predictions')(x)
model = Model(img_input, [x], name='')

Tensor("max_pooling2d_1/MaxPool:0", shape=(?, 14, 14, 128), dtype=float32)
Tensor("conv2d_1/BiasAdd:0", shape=(?, 14, 14, 128), dtype=float32)


In [9]:
model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_images, train_labels, epochs=1, batch_size=64)

Epoch 1/1
60000/60000 [==============================] - 49s 822us/step - loss: 0.1359 - acc: 0.9587
